In [ ]:
from __future__ import division, print_function, absolute_import

import tflearn
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.normalization import local_response_normalization
from tflearn.layers.estimator import regression

import numpy as np
import pdb
import os


In [2]:
# Data loading and preprocessing
X = np.genfromtxt('Xtrain.txt').T
X = X.reshape([-1, 128, 128, 1])
Y = np.genfromtxt('Ytrain.txt').T
Y = Y.reshape([-1,10])
testX = np.genfromtxt('Xdev.txt').T
testX = testX.reshape([-1, 128, 128, 1])
testY = np.genfromtxt('Ydev.txt').T
testY = testY.reshape([-1,10])

In [3]:
#Conv Layer Parameters
keep_prob = 1
filter_depth = [64,128,256,512]
filter_sizes = [2 , 2,  2,  2]
fully_connected_size = 512
activation = 'elu'
regularizer = 'L2'
weights_init = "Xavier"
weight_decay = 0.0001
MaxPoolLayer = ['True','True','True','True']
MaxPoolSize  = [2, 2, 2, 2]
# Building convolutional network
network = input_data(shape=[None, 128, 128, 1], name='input')

#Convolutional Layers
for i in range(len(filter_sizes)):
    network = conv_2d(network, filter_depth[i], filter_sizes[i], strides=2, activation=activation, weights_init=weights_init, regularizer=regularizer, weight_decay=weight_decay)
    network = max_pool_2d(network, MaxPoolSize[i])
    network = dropout(network, keep_prob)
    #network = local_response_normalization(network)
#Fully Connected Layer
network = fully_connected(network, 1024, activation='elu')
network = dropout(network, keep_prob)
#Output Layer
network = fully_connected(network, 10, activation='softmax')
network = regression(network, optimizer='adam', learning_rate=0.001, loss='categorical_crossentropy', name='target')

Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [4]:
# Training
model = tflearn.DNN(network, tensorboard_verbose=0, tensorboard_dir='/home/ubuntu/logs/', best_checkpoint_path='/home/ubuntu/logs/')
model.fit({'input': X}, {'target': Y}, n_epoch=25, validation_set=({'input': testX}, {'target': testY}), batch_size = 256, snapshot_step=100, snapshot_epoch=True, show_metric=True, run_id='Model1')
model.save("BaseModel1.tflearn")

Training Step: 949  | total loss: 0.14909 | time: 64.337s
| Adam | epoch: 025 | loss: 0.14909 - acc: 0.9509 -- iter: 9472/9500
Training Step: 950  | total loss: 0.14406 | time: 67.155s
| Adam | epoch: 025 | loss: 0.14406 - acc: 0.9507 | val_loss: 0.78393 - val_acc: 0.7760 -- iter: 9500/9500
--
INFO:tensorflow:/home/ubuntu/BaseModel1.tflearn is not in all_model_checkpoint_paths. Manually adding it.
